In [0]:

/*
============================================================
Stored Procedure: Load Silver Layer (Bronze => Silver)
============================================================
Script Purpose:
  This stored procedure loads data into the 'silver' schema from bronze layer.
  It performs the following actions:
    - Truncates the silver tables before loading the data.
    - Uses some transformations before loading the data directly to make the data more clear

Parameters:
  - None.
  This stored procedure doesn't accept any parameters or return any values

Usage Example:
  -  CALL datawarehouse.silver.load_silver();
*/
call datawarehouse.silver.load_silver();

CREATE OR REPLACE PROCEDURE datawarehouse.silver.load_silver()
LANGUAGE SQL
SQL SECURITY INVOKER 
COMMENT 'Loads data from bronze to silver with transformations'
AS 

BEGIN
  -- ====================
  -- crm_cust_info Table
  -- ====================
  -- Remove the Duplicates and Nulls from Primary_key (cust_id)

  truncate table datawarehouse.silver.crm_cust_info;
  INSERT INTO datawarehouse.silver.crm_cust_info (
    cst_id,
    cst_key,
    cst_firstname,
    cst_lastname,
    cst_marital_status,
    cst_gndr,
    cst_create_date
  )
  select 
  cst_id, 
  cst_key,
  TRIM(cst_firstname) AS cst_firstname,
  TRIM(cst_lastname) AS cst_lastname,

  CASE 
    WHEN upper(trim(cst_marital_status)) = 'S' then 'Single'
    WHEN UPPER(trim(cst_marital_status)) = 'M' then 'Married'
    ELSE 'N/A'
  END AS cst_marital_status,

  CASE
    WHEN cst_gndr = 'F' THEN 'Female'
    WHEN cst_gndr = 'M' THEN 'Male'
    ELSE 'N/A'
  END AS cst_gndr,
  cst_create_date
  from (
    select 
      *, 
      row_number() over (partition by cst_id order by cst_create_date desc) flag_last
    from datawarehouse.bronze.crm_cust_info
  ) where flag_last = 1;




  -- =========================================
  -- crm_prd_info Table
  -- =========================================
  -- Transforming the crm_prd_info table for bronze to silver
  truncate table datawarehouse.silver.crm_prd_info;
  insert into datawarehouse.silver.crm_prd_info (
    prd_id,
    cat_id,
    prd_key,
    prd_nm,
    prd_cost,
    prd_line,
    prd_start_dt,
    prd_end_dt
  )
  select 
  prd_id,
  replace(substr(prd_key, 1, 5), '-', '_') as cat_id,
  substr(prd_key, 7, len(prd_key)) as prd_key,
  prd_nm,
  coalesce(prd_cost, 0) as prd_cost,
  case upper(trim(prd_line)) 
    when 'M' then 'Mountain'
    when 'R' then 'Road'
    when 'S' then 'Other Sales'
    when 'T' then 'Touring'
    else 'n/a'
  end as prd_line,
  prd_start_dt,
  lead(prd_start_dt) OVER (partition by prd_key order by prd_start_dt) as prd_end_dt
  from datawarehouse.bronze.crm_prd_info;



  -- =========================================
  -- crm_sales_details Table
  -- =========================================
  -- Transforming the crm_sales_details table for bronze to silver
  truncate table datawarehouse.silver.crm_sales_details;
  insert into datawarehouse.silver.crm_sales_details (
    sls_ord_num,
    sls_prd_key,
    sls_cust_id,
    sls_order_dt,
    sls_ship_dt,
    sls_due_dt,
    sls_sales,
    sls_quantity,
    sls_price
  ) 
  select 
  sls_ord_num, 
  sls_prd_key,
  sls_cust_id,
  case 
    when sls_order_dt = 0 or len(sls_order_dt) != 8 then null
    else try_to_date(sls_order_dt, 'yyyyMMdd') 
  end as sls_order_dt,
  case 
    when sls_ship_dt = 0 or len(sls_ship_dt) != 8 then null
    else try_to_date(sls_ship_dt, 'yyyyMMdd') 
  end as sls_ship_dt,
  case 
    when sls_due_dt = 0 or len(sls_due_dt) != 8 then null
    else try_to_date(sls_due_dt, 'yyyyMMdd') 
  end as sls_due_dt,

  case 
  when sls_sales is null or sls_sales <= 0 
    then sls_quantity * abs(sls_price)
  else sls_sales
  end as sls_sales,

  sls_quantity,

  case 
  when sls_price is null or sls_price <= 0 
    then sls_sales / nullif(sls_quantity, 0)
  else sls_price
  end as sls_price

  from datawarehouse.bronze.crm_sales_details;


  -- select * from datawarehouse.silver.crm_sales_details;


  -- =========================================
  -- crm_prd_info Table
  -- =========================================
  -- Transforming the crm_prd_info table for bronze to silver
  TRUNCATE TABLE datawarehouse.silver.erp_cust_az12;
  INSERT INTO datawarehouse.silver.erp_cust_az12 (
    CID,
    BDATE,
    GEN
  )
  select 
  CASE 
    WHEN cid LIKE 'NAS%' THEN SUBSTR(cid, 4, len(cid))
    ELSE cid
  END AS cid,

  CASE 
    WHEN bdate > current_date() THEN null
    ELSE bdate
  END AS bdate,

  CASE
      WHEN upper(trim(gen)) IN ('F', 'FEMALE') THEN 'Female'
      WHEN upper(trim(gen)) IN ('M', 'MALE') THEN 'Male'
      ELSE 'n/a'
    END AS gen

  from datawarehouse.bronze.erp_cust_az12;

  -- select * from datawarehouse.silver.erp_cust_az12;


  -- =========================================
  -- erp_loc_a101 Table
  -- =========================================
  -- Transforming the erp_loc_a101 table for bronze to silver
  truncate table datawarehouse.silver.erp_loc_a101;
  insert into datawarehouse.silver.erp_loc_a101 (
    cid,
    cntry
  )
  select 
  replace(cid, '-', '') as cid,
  case 
    when upper(trim(cntry)) in ('US', 'USA', 'UNITED STATES') then 'United States'
    when upper(trim(cntry)) in ('CA', 'CANADA') then 'Canada'
    when upper(trim(cntry)) in ('DE', 'GERMANY') then 'Germany'
    when upper(trim(cntry)) in ('AU', 'AUSTRALIA') then 'Australia'
    when upper(trim(cntry)) in ('FR', 'FRANCE') then 'France'
    when upper(trim(cntry)) in ('UK', 'UNITED KINGDOM') then 'United Kingdom'
    else 'n/a'
  end as cntry
  from datawarehouse.bronze.erp_loc_a101;



  -- =========================================
  -- erp_loc_a101 Table
  -- =========================================
  -- Transforming the erp_loc_a101 table for bronze to silver
  truncate table datawarehouse.silver.erp_px_cat_g1v2;
  insert into datawarehouse.silver.erp_px_cat_g1v2
  (
    id, cat, SUBCAT, MAINTENANCE
  )
  select 
  id, cat, SUBCAT, MAINTENANCE
  from datawarehouse.bronze.erp_px_cat_g1v2;

  -- select * from datawarehouse.silver.erp_px_cat_g1v2;

END





